In [1]:
import numpy as np
import pandas as pd
import warnings
import time
import astropy.units as u   #单位
from astropy.coordinates import SkyCoord   # 天球坐标
from astroquery.irsa import Irsa  # 所需数据库 

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [2]:
# 读入现有数据

data = pd.read_csv("galaxy_all.csv", usecols=[0,1,2,3,6], skiprows=1, header=None, 
                   names=['name', 'band', 'ra', 'dec', 'herf_70'])
data['herf_100'] = np.nan
data['herf_160'] = np.nan
data['herf_250'] = np.nan
data['herf_350'] = np.nan
data['herf_500'] = np.nan
data['wisevega_1'] = np.nan
data['wisevega_2'] = np.nan
data['wisevega_3'] = np.nan
data['wisevega_4'] = np.nan
data['2mvega_j'] = np.nan
data['2mvega_h'] = np.nan
data['2mvega_k'] = np.nan

# 'herf_100', 'herf_160','herf_250', 'herf_350', 'herf_500', 
# 'wisevega_1', 'wisevega_2', 'wisevega_3', 'wisevega_4', '2mvega_j', '2mvega_h', '2mvega_k'

In [3]:
# 寻找所需星等/流量数据

index = len(data)

for i in range(1601, index):  # 修改range初值，可从上次中断保存处继续运行
    print ('The echo is', i, '/', index)
    ra = data['ra'][i]
    dec = data['dec'][i]
    warnings.filterwarnings('ignore')
    
    # herschel 100 微米
    table_her100 = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                     catalog='ppsc_100', radius=3 * u.arcsec)
    if len(table_her100) > 0:
        data['herf_100'][i] = table_her100['flux'][0]
    
    # herschel 160 微米
    table_her160 = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                     catalog='ppsc_160', radius=3 * u.arcsec)  
    if len(table_her160) > 0:
        data['herf_160'][i] = table_her160['flux'][0]
    
    # herschel 250 微米
    table_her250 = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                     catalog='spsc250', radius=3 * u.arcsec)
    if len(table_her250) > 0:
        data['herf_250'][i] = table_her250['flux'][0]
    
    # herschel 350 微米
    table_her350 = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                     catalog='spsc350', radius=3 * u.arcsec)
    if len(table_her350) > 0:
        data['herf_350'][i] = table_her350['flux'][0]
    
    # herschel 500 微米
    table_her500 = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                     catalog='spsc500', radius=3 * u.arcsec)
    if len(table_her500) > 0:
        data['herf_500'][i] = table_her500['flux'][0]
    
    # wise：w1mpro(3.4微米), w2mpro(4.2微米), w3mpro(12微米), w4mpro(22微米)
    table_wise = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                   catalog='allwise_p3as_psd', radius=3 * u.arcsec)
    if len(table_wise) > 0:
        data['wisevega_1'][i] = table_wise['w1mpro'][0]
        data['wisevega_2'][i] = table_wise['w2mpro'][0]
        data['wisevega_3'][i] = table_wise['w3mpro'][0]
        data['wisevega_4'][i] = table_wise['w4mpro'][0]
    
    # 2mass：j_m(1.23微米), h_m(1.66微米), k_m(2.15微米)
    table_2mass = Irsa.query_region(SkyCoord(ra, dec, unit=(u.deg, u.deg)), 
                                    catalog='fp_psc', radius=3 * u.arcsec)
    if len(table_2mass) > 0:
        data['2mvega_j'][i] = table_2mass['j_m'][0]
        data['2mvega_h'][i] = table_2mass['h_m'][0]
        data['2mvega_k'][i] = table_2mass['k_m'][0]
    
    '''
    # 定时中断保存
    interval = 200
    if (i % interval) == 0 and i != 0 :   # 每查询200次保存一次文件
        data_name = 'flux_original_%s.csv' % (i)
        data.to_csv(data_name, float_format="%.6f", index=0)
        print ('The file %s has been saved.\n' % data_name)
        time.sleep(5)   # 提示信息延时5秒
    '''    
data.to_csv('flux_original.csv', float_format="%.6f", index=0)